In [1]:
# Imports
import os
import subprocess
import sys

# REMOVE!!
from ir_utils import *

# model

from ir_lmart import *

# HPO

from hpo import *
from HpoWorker import *

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

# from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS
from hpbandster.examples.commons import MyWorker
import random

# In[2]:


# In[3]:

In [2]:
def tickets(max):
    return {i:i for i in range(1,max)}

In [3]:
million_tickets = tickets(1000000)

In [4]:
len(million_tickets)

999999

In [5]:
def get_train_budget_data_file(budget, query_list, train_data_file):
    # Budget is percentage of training data: 
    # min_budget = 10%
    # max_budget = 100%
    if (int(budget) <= 100 or int(budget) >= 10):
        len_queries = len(query_list)
        budgeted_queries = round(len_queries * (budget / 100))
        print('total budget:', len_queries)
        print('allocated budget:', budgeted_queries)
        train_budget_queries_file = train_data_file + '_budget' + str(budget)
        if not os.path.exists(train_budget_queries_file):
            with open(train_data_file, 'rt') as f_in:
                with open(train_budget_queries_file, 'wt') as budget_file_out:
                    for query_feature in f_in:                        
                        qid = query_feature.split()[1].split(':')[1]
#                         print(qid)
                        if qid in query_list[0:budgeted_queries]:
                            
                            budget_file_out.write(query_feature)
        else:
            print("File already exists")
            return train_budget_queries_file                
    else:
        print('Budget is outside the limits (10% < b < 100%): ', budget)
        return 

In [6]:
# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        
# In[4]:

In [7]:
class fakeParser:
    def __init__(self):
        self.min_budget = 1
        self.max_budget = 100
        self.n_iterations = 20
        self.n_workers = 1
        self.hpo_method = 'rs'
        

In [8]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=int, help='Minimum budget used during the optimization.',    default=10)
    parser.add_argument('--max_budget',   type=int, help='Maximum budget used during the optimization.',    default=100)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)

#     args=parser.parse_args()
    args = fakeParser()
    hpo_method = args.hpo_method
#     dataset = sys.argv[1] # 'bioasq'
#     workdir = './' + dataset + '_dir/'
#     data_split = sys.argv[2] # 'test'

    dataset = 'bioasq'
    workdir = './' + dataset + '_dir/'
    data_split =  'train'
    k_fold = 's1' 
    ranklib_location = '../../../ranklib/'
    
    
    
#     train_data_file = './bioasq_dir/bioasq.trai_features_reduced'
#     val_data_file = './bioasq_dir/bioasq.dev_features_reduced'
#     test_data_file = './bioasq_dir/bioasq.test_features_reduced'
    
    train_data_file = './bioasq_dir/bioasq_train_features'
    val_data_file = './bioasq_dir/bioasq_dev_features'
    test_data_file = './bioasq_dir/bioasq_test_features'
    
    l2r_model = '_lmart_'
    

In [9]:
    confdir = './' + dataset + '_config/'
    enabled_features_file = confdir + dataset + l2r_model + 'enabled_features'
    
    
    metric2t = 'MAP' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'
    
    ranker_type = '6' # LambdaMART
    
    # normalization: Feature Engineering?
    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    l2r_params = [
        '-validate',
        val_data_file,
        '-ranker',
        ranker_type,
        '-metric2t',
        metric2t,
        '-feature',
        enabled_features_file
    ]
    
    # Run train
    
#     lmart_model = L2Ranker(ranklib_location, l2r_params)
    lmart_model = L2Ranker(ranklib_location, l2r_params, norm_params)
    


In [10]:
trec_eval_command = '../../eval/trec_eval'
qrels_val_file = './bioasq_dir/bioasq_dev_qrels'


In [11]:
# Start a nameserver (see example_3)


if hpo_method == 'rs':
    hpo_run_id = "RandomSearch"
elif hpo_method == 'bohb':
    hpo_run_id = "BOHB"

NS = hpns.NameServer(run_id = hpo_run_id, host='127.0.0.1', port=None)
NS.start()


# In[12]:

save_model_prefix = workdir + 'ejemplo_model'
run_file_prefix = workdir + 'run_ejemplo'

train_queries_file = '../../bioasq_data/bioasq.' + 'train' + '.json'
            
query_list = load_queries(train_queries_file)
qid_list = [q['id'] for q in query_list] 



len_queries = len(qid_list) 
real_max_budget = round((len_queries * args.max_budget) / 100)
real_min_budget = round((len_queries * args.min_budget) / 100)

# train_features_file =  fold_dir + dataset + '_' + 'train' + '_features'
train_features_file =  workdir + dataset + '_' + 'train' + '_features'


budget_train_features_file = train_data_file
# budget_train_features_file = workdir + 'budget_ejemplo'


workers=[]
for i in range(args.n_workers):
    worker = HpoWorker(lmart_model, save_model_prefix, run_file_prefix, budget_train_features_file, qid_list, trec_eval_command, 
                       qrels_val_file, million_tickets, nameserver='127.0.0.1',run_id=hpo_run_id, id=i)
    worker.run(background=True)
    workers.append(worker)


# In[13]:

In [12]:
# # Run an optimizer (see example_2)


In [13]:
# Random search

if hpo_method == 'rs':
    rs = RS(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = real_max_budget, max_budget = real_max_budget
                   )
    res = rs.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)

    rs.shutdown(shutdown_workers=True)
elif hpo_method == 'bohb':
    bohb = BOHB(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = real_min_budget, max_budget = real_max_budget
                   )
    res = bohb.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)
    bohb.shutdown(shutdown_workers=True)


# In[14]:


# Step 4: Shutdown
# After the optimizer run, we must shutdown the master and the nameserver.



Model saved:  ./bioasq_dir/ejemplo_model_id1_budget1751_leaves20_lr0.42_n600
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id2_budget1751_leaves100_lr0.17_n1150
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id3_budget1751_leaves5_lr0.48_n2000
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id4_budget1751_leaves25_lr0.14_n2000
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id5_budget1751_leaves65_lr0.43_n1950
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id6_budget1751_leaves50_lr0.5_n800
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id7_budget1751_leaves70_lr0.19_n1350
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id8_budget1751_leaves95_lr0.32_n1250
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id9_budget1751_leaves35_lr0.14_n1550
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id10_budget1751_leaves95_lr0.11_n850
Run error:  None
Model saved:  ./bioasq_dir/ejemplo_model_id11_budget1

In [14]:

NS.shutdown()


# In[15]:


# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()


# In[16]:


all_runs = res.get_all_runs()



print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in all_runs])/args.max_budget))
print('The run took  %.1f seconds to complete.'%(all_runs[-1].time_stamps['finished'] - all_runs[0].time_stamps['started']))


# In[18]:


#     qrels_test_file = './bioasq_dir/bioasq.test_qrels'
#     run_val_file = './this.file'
#     lmart_model = res['info']
#     lmart_model.gen_run_file(test_data_file, run_val_file)
#     eval(trec_eval_command, qrels_test_file, run_val_file)


# In[10]:

Best found configuration: {'learning_rate': 0.07, 'n_leaves': 45, 'n_trees': 1600}
A total of 20 unique configurations where sampled.
A total of 20 runs where executed.
Total budget corresponds to 350.2 full function evaluations.
The run took  768.7 seconds to complete.


In [15]:
res.get_incumbent_id()

(11, 0, 0)

In [16]:
id2config[res.get_incumbent_id()]

{'config': {'learning_rate': 0.07, 'n_leaves': 45, 'n_trees': 1600},
 'config_info': {}}

In [17]:
# res.get_learning_curves()

In [18]:
res.get_all_runs()

[config_id: (0, 0, 0)	budget: 1751.000000	loss: 0.5718
 time_stamps: 0.0005159378051757812 (submitted), 0.004738330841064453 (started), 36.656166791915894 (finished)
 info: {'map': '0.4282', 'P_20': '0.2650', 'ndcg_cut_20': '0.5333'},
 config_id: (1, 0, 0)	budget: 1751.000000	loss: 0.5651999999999999
 time_stamps: 36.65819764137268 (submitted), 36.65831446647644 (started), 86.41748905181885 (finished)
 info: {'map': '0.4348', 'P_20': '0.2655', 'ndcg_cut_20': '0.5446'},
 config_id: (2, 0, 0)	budget: 1751.000000	loss: 0.567
 time_stamps: 86.4191586971283 (submitted), 86.41925644874573 (started), 104.33919620513916 (finished)
 info: {'map': '0.4330', 'P_20': '0.2600', 'ndcg_cut_20': '0.5357'},
 config_id: (3, 0, 0)	budget: 1751.000000	loss: 0.5606
 time_stamps: 104.34061002731323 (submitted), 104.34068608283997 (started), 142.93805027008057 (finished)
 info: {'map': '0.4394', 'P_20': '0.2600', 'ndcg_cut_20': '0.5409'},
 config_id: (4, 0, 0)	budget: 1751.000000	loss: 0.5710999999999999
 ti

In [19]:
res.get_id2config_mapping()

{(0, 0, 0): {'config': {'learning_rate': 0.42, 'n_leaves': 20, 'n_trees': 600},
  'config_info': {}},
 (1,
  0,
  0): {'config': {'learning_rate': 0.17,
   'n_leaves': 100,
   'n_trees': 1150}, 'config_info': {}},
 (2, 0, 0): {'config': {'learning_rate': 0.48, 'n_leaves': 5, 'n_trees': 2000},
  'config_info': {}},
 (3,
  0,
  0): {'config': {'learning_rate': 0.14,
   'n_leaves': 25,
   'n_trees': 2000}, 'config_info': {}},
 (4,
  0,
  0): {'config': {'learning_rate': 0.43,
   'n_leaves': 65,
   'n_trees': 1950}, 'config_info': {}},
 (5, 0, 0): {'config': {'learning_rate': 0.5, 'n_leaves': 50, 'n_trees': 800},
  'config_info': {}},
 (6,
  0,
  0): {'config': {'learning_rate': 0.19,
   'n_leaves': 70,
   'n_trees': 1350}, 'config_info': {}},
 (7,
  0,
  0): {'config': {'learning_rate': 0.32,
   'n_leaves': 95,
   'n_trees': 1250}, 'config_info': {}},
 (8,
  0,
  0): {'config': {'learning_rate': 0.14,
   'n_leaves': 35,
   'n_trees': 1550}, 'config_info': {}},
 (9, 0, 0): {'config': {'lea